In [1]:
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd

In [2]:
df = pd.read_csv('../data/MNIST_train_small.csv', header = None)
df.head(5)
Y = df.iloc[:, 0]
X = df.iloc[:, 1:]

In [3]:
df2 = pd.read_csv('../data/MNIST_test_small.csv', header = None)
df2.head(5)
Y2 = df2.iloc[:, 0]
X2 = df2.iloc[:, 1:]

In [27]:
distances = cdist(XA=X, XB=X, metric = 'minkowski', p =2.0)

In [28]:
labels = np.tile(Y, (X.shape[0], 1))

In [29]:
sorted_points_indices = np.apply_along_axis(np.argsort, 1, distances)

In [30]:
sorted_points_indices

array([[   0,  429,  773, ..., 2052, 2080, 1561],
       [   1,  739,  164, ...,  311,  548,  935],
       [   2, 2598, 2136, ...,  123, 1080, 2747],
       ...,
       [2997, 2399, 2786, ...,  123, 1080, 2747],
       [2998, 2395, 1620, ...,  848, 2747, 1080],
       [2999, 1410, 2214, ...,  848,  935, 1080]], dtype=int64)

In [31]:
sorted_distances = distances[np.arange(sorted_points_indices.shape[0])[:, None], sorted_points_indices]

In [32]:
sorted_labels = labels[np.arange(sorted_points_indices.shape[0])[:, None], sorted_points_indices]

In [33]:
k_sorted_labels = sorted_labels[:, :2]
k_sorted_distances = sorted_distances[:, :2]

In [34]:
supermatrix = np.zeros((X.shape[0], 2, 2))
supermatrix[:, :, 0] = k_sorted_distances
supermatrix[:, :, 1] = k_sorted_labels

In [37]:
def majority(x, k):
    """
    Auxiliary function to get the majority among k-nearest neighbors. It is extended to add custom tie-breaking rule
    (instead of standard picking first of satisfying numbers): weighted k-nearest neighbors (the class with smallest
    sum of distances to target)

     :param x: row, where first k cells are distances to k-nearest neighbors and last k cells are labels of k-nearest
     neigbors

     :return: value of label
    """
    distances = x[:k]
    labels = x[k:]
    values, counts = np.unique(x, return_counts = True)
    if k == 1:
        return labels[0]
    elif (k>1) and (list(counts).count(max(list(counts))) > 1):
        labels_to_compare = values[np.where(counts == max(list(counts)))]
        weighted_dist = []
        for i in range(labels_to_compare.shape[0]):
            weighted_dist.append(np.sum(distances[np.where(labels == labels_to_compare[i])]))
        return labels_to_compare[np.argmin(weighted_dist)]
    else:
        return max(set(list(labels)), key = list(labels).count)

In [38]:
supermatrix.reshape((supermatrix.shape[0],2 * supermatrix.shape[1]),order='F')

array([[0.00000000e+00, 1.30419247e+03, 6.00000000e+00, 6.00000000e+00],
       [0.00000000e+00, 1.34150624e+03, 3.00000000e+00, 3.00000000e+00],
       [0.00000000e+00, 1.04427678e+03, 9.00000000e+00, 4.00000000e+00],
       ...,
       [0.00000000e+00, 1.89143861e+03, 8.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 7.91513108e+02, 1.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 8.41715510e+02, 7.00000000e+00, 7.00000000e+00]])

In [41]:
predicted_labels = np.apply_along_axis(majority, 1,
                                       supermatrix.reshape((supermatrix.shape[0],
                                                                         2 * supermatrix.shape[1]),
                                                                        order='F'), k=2)

In [42]:
predicted_labels

array([6., 3., 0., ..., 0., 1., 7.])

In [45]:
supermatrix.reshape((supermatrix.shape[0],2 * supermatrix.shape[1]), order='F')[2]

array([   0.        , 1044.27678323,    9.        ,    4.        ])

In [46]:
majority(supermatrix.reshape((supermatrix.shape[0],2 * supermatrix.shape[1]), order='F')[2], 2)

0.0

In [48]:
x=supermatrix.reshape((supermatrix.shape[0],2 * supermatrix.shape[1]), order='F')[2]
k=2

In [49]:
distances = x[:k]
labels = x[k:]

In [50]:
distances

array([   0.        , 1044.27678323])

In [51]:
labels

array([9., 4.])

In [54]:
values, counts = np.unique(labels, return_counts = True)

In [55]:
values

array([4., 9.])

In [56]:
counts

array([1, 1], dtype=int64)

In [57]:
(k>1) and (list(counts).count(max(list(counts))) > 1)

True

In [ ]:
labels_to_compare = values[np.where(counts == max(list(counts)))]
        weighted_dist = []
        for i in range(labels_to_compare.shape[0]):
            weighted_dist.append(np.sum(distances[np.where(labels == labels_to_compare[i])]))
labels_to_compare[np.argmin(weighted_dist)]